## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-06-51-00 +0000,wsj,New Zealand’s Central Bank Keeps Rates on Hold...,https://www.wsj.com/articles/new-zealands-cent...
1,2025-07-09-06-47-57 +0000,nyt,Guatemala Closes Schools in Capital After Eart...,https://www.nytimes.com/2025/07/09/world/ameri...
2,2025-07-09-06-44-30 +0000,bbc,Housebuilders to pay £100m to avoid legal deci...,https://www.bbc.com/news/articles/c2d0740px42o
3,2025-07-09-06-21-45 +0000,bbc,Seoul returns six North Koreans with 'strong d...,https://www.bbc.com/news/articles/cwyrwp8r1xgo
4,2025-07-09-06-06-36 +0000,bbc,Australian childcare operator to install CCTV ...,https://www.bbc.com/news/articles/c3vdel6y5n2o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2292/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,16
61,tariffs,8
195,uk,5
271,tariff,5
0,new,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
32,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,44
8,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,43
65,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...,38
67,2025-07-08-13-44-50 +0000,nyt,Trump Targets Japan With 25% Tariffs After Sev...,https://www.nytimes.com/2025/07/08/business/tr...,37
36,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,35


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
32,44,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
14,28,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
8,19,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
33,19,2025-07-08-21-31-55 +0000,bbc,King and Macron toast 'ever closer' UK-France ...,https://www.bbc.com/news/articles/cvg87y6d5j4o
16,17,2025-07-09-00-11-42 +0000,bbc,Moment house is swept away in New Mexico flash...,https://www.bbc.com/news/videos/cren2zq1y3qo
36,17,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...
82,15,2025-07-08-10-33-24 +0000,bbc,Israeli defence minister plans to move Gaza's ...,https://www.bbc.com/news/articles/c8rp31lk7mzo
20,14,2025-07-08-23-44-53 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo
50,14,2025-07-08-16-41-29 +0000,bbc,Imposter used AI to pose as Marco Rubio and co...,https://www.bbc.com/news/articles/crrqkyyjewno
47,14,2025-07-08-17-12-03 +0000,nyt,The French Seaside Factory Trying to Break Chi...,https://www.nytimes.com/2025/07/08/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
